## DASK_ML

In [1]:
import dask.dataframe as dd
from dask.distributed import Client
import pandas as pd
import numpy as np

In [2]:
# Initialize Dask client
client = Client(n_workers=4, threads_per_worker=1, memory_limit='2GB')

In [3]:
# Load multiple Parquet files
data = dd.read_parquet('parquet_data/*.parquet')

# print all columns and their data types
print(data.dtypes)

Summons Number                       string[pyarrow]
Plate ID                             string[pyarrow]
Registration State                   string[pyarrow]
Plate Type                           string[pyarrow]
Issue Date                           string[pyarrow]
Violation Code                       string[pyarrow]
Vehicle Body Type                    string[pyarrow]
Vehicle Make                         string[pyarrow]
Issuing Agency                       string[pyarrow]
Street Code1                         string[pyarrow]
Street Code2                         string[pyarrow]
Street Code3                         string[pyarrow]
Vehicle Expiration Date              string[pyarrow]
Violation Location                   string[pyarrow]
Violation Precinct                   string[pyarrow]
Issuer Precinct                      string[pyarrow]
Issuer Code                          string[pyarrow]
Issuer Command                       string[pyarrow]
Issuer Squad                         string[py

### Data Cleaning

In [4]:
# keep only the columns we need
columns_to_keep = [
    'Violation County',
    'Issue Date',
    'Violation Time',]

data = data[columns_to_keep]

# rename columns to remove spaces and make them lowercase
data = data.rename(columns={
    'Violation County': 'violation_county',
    'Issue Date': 'issue_date',
    'Violation Time': 'violation_time'
})

# fix the county names
remap_county_dict = {
    'K' : 'Brooklyn',
    'Q' : 'Queens',
    'NY': 'Manhattan',
    'QN': 'Queens',
    'BK': 'Brooklyn',
    'R' : 'Staten Island',
    'BX': 'Bronx',
    'ST': 'Staten Island',
    'MN': 'Manhattan',
    'KINGS': 'Brooklyn',
    'QNS': 'Queens',
    'BRONX': 'Bronx',
    'RICHM': 'Staten Island',
    'RICH': 'Staten Island',
    'Queens': 'Queens',
    'Manhattan': 'Manhattan',
    'Bronx': 'Bronx',
    'Brooklyn': 'Brooklyn',
    'Staten Island': 'Staten Island',
    'Rich': 'Staten Island',
    'QUEEN': 'Queens',
    'NEW Y': 'Manhattan',
    'NYC': 'Manhattan',
    'USA': 'Unknown',
    'K   F': 'Brooklyn',
    'VINIS': 'Staten Island',
    'NEWY': 'Manhattan',
    'KING': 'Brooklyn',
    'PBX': 'Bronx',
    'MS': 'Unknown',
    'ABX': 'Bronx',
    '103': 'Unknown',
    '108': 'Unknown',
    'MH': 'Manhattan',
    'MAN': 'Manhattan',
    'P': 'Unknown',
    'N': 'Unknown',
    'A': 'Unknown',
    'F': 'Unknown',
    'QU': 'Queens'
}
data['violation_county'] = data['violation_county'].map(remap_county_dict).astype('category')

# convert the Issue Date to a datetime object
data['issue_date'] = dd.to_datetime(data['issue_date'], format='%m/%d/%Y', errors='coerce')

# Remove 'A' and 'P' from the end of the time, add ' AM' or ' PM' accordingly
data['violation_time'] = data['violation_time'].str.slice(stop=-1) + ' ' + data['violation_time'].str.slice(start=-1).replace({'A': 'AM', 'P': 'PM'})

# Convert the Violation Time to a datetime object
data['violation_time'] = dd.to_datetime(data['violation_time'], format='%I%M %p', errors='coerce')

# create a new column for the day of the week the violation was issued
data['violation_day_week'] = data['issue_date'].dt.dayofweek

# create a new column for the day of the month the violation was issued
data['violation_day_month'] = data['issue_date'].dt.day

# create a new column for the month the violation was issued
data['violation_month'] = data['issue_date'].dt.month

# create a new column for the year the violation was issued
data['violation_year'] = data['issue_date'].dt.year

# keep only rows with valid year (2013-2024)
data = data[(data['violation_year'] >= 2013) & (data['violation_year'] <= 2024)]

# keep only rows with valid month (1-12)
data = data[(data['violation_month'] >= 1) & (data['violation_month'] <= 12)]

# drop all rows with missing values
data = data.dropna()

# create a new column for the hour of the day the violation was issued
data['violation_hour'] = data['violation_time'].dt.hour.astype('int32')

# drop the Issue Date and Violation Time columns
data = data.drop(columns=['violation_time'])

/home/drew99/School/BigData/project/venv-bigdproject/lib/python3.10/site-packages/dask_expr/_collection.py:4160: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('violation_county', 'object'))

  warnings.warn(meta_warning(meta))


In [5]:
# drop county column
data = data.drop(columns=['violation_county'])

In [6]:
#print type of columns
print(data.dtypes)

issue_date             datetime64[ns]
violation_day_week              int32
violation_day_month             int32
violation_month                 int32
violation_year                  int32
violation_hour                  int32
dtype: object


In [7]:
# print head of the data
# print(data.head())

In [8]:
# reduce the dataset to an hourly level - count the number of violations per hour
data = data.groupby(['violation_year', 'violation_month', 'violation_day_month', 'violation_day_week', 'violation_hour']).size().reset_index()
data = data.rename(columns={0: 'violation_count'})

In [9]:
print(data.dtypes)

violation_year         int32
violation_month        int32
violation_day_month    int32
violation_day_week     int32
violation_hour         int32
violation_count        int64
dtype: object


In [10]:
# make new column of format ='%Y-%m-%d %H:%M:%S' for the datetime (for joining with weather data)
data['datetime'] = data['violation_year'].astype(str) + '-' + data['violation_month'].astype(str).str.zfill(2) + '-' + data['violation_day_month'].astype(str).str.zfill(2) + ' ' + data['violation_hour'].astype(str).str.zfill(2) + ':00:00'
data['datetime'] = dd.to_datetime(data['datetime'], format='%Y-%m-%d %H:%M:%S')

In [11]:
print(data.dtypes)

violation_year                  int32
violation_month                 int32
violation_day_month             int32
violation_day_week              int32
violation_hour                  int32
violation_count                 int64
datetime               datetime64[ns]
dtype: object


In [12]:
# Group the data by month and count the number of violations
monthly_violations = data.groupby(data['datetime'].dt.to_period('M')).size()

In [13]:
# Plot the monthly violations
# monthly_violations.compute().plot()

In [14]:
# from the data take only the rows that have the datetime between 2013-01-01 and 2015-03-31
data = data[(data['datetime'] >= '2013-01-01') & (data['datetime'] <= '2015-03-31')]

In [17]:
print(data.dtypes)

violation_year                  int32
violation_month                 int32
violation_day_month             int32
violation_day_week              int32
violation_hour                  int32
violation_count                 int64
datetime               datetime64[ns]
dtype: object


In [18]:
# Step 1: Create a complete range of dates and hours directly with hourly frequency
# min_date = data['datetime'].min().compute()
# max_date = data['datetime'].max().compute()
min_date = '2013-01-01 00:00:00'
min_date = pd.to_datetime(min_date)
max_date = '2015-03-31 23:00:00'
max_date = pd.to_datetime(max_date)
print(f"Taking data from {min_date} to {max_date}")
all_date_hours = pd.date_range(start=min_date, end=max_date, freq='h').to_frame(index=False, name='datetime')

# Extract year, month, day_of_month, and day_of_week directly from the datetime
all_date_hours['violation_year'] = all_date_hours['datetime'].dt.year
all_date_hours['violation_month'] = all_date_hours['datetime'].dt.month
all_date_hours['violation_day_month'] = all_date_hours['datetime'].dt.day
all_date_hours['violation_day_week'] = all_date_hours['datetime'].dt.dayofweek
all_date_hours['violation_hour'] = all_date_hours['datetime'].dt.hour

# Step 2: Convert the comprehensive Pandas DataFrame to a Dask DataFrame
all_date_hours_dd = dd.from_pandas(all_date_hours, npartitions=10)

# Step 3: Merge with existing data using Dask's merge function
complete_data = dd.merge(all_date_hours_dd, data, on=['violation_year', 'violation_month', 'violation_day_month', 'violation_day_week', 'violation_hour'], how='outer')

# Fill missing values with 0
complete_data['violation_count'] = complete_data['violation_count'].fillna(0).astype(int)

Taking data from 2013-01-01 00:00:00 to 2015-03-31 23:00:00


In [19]:
# drop datetime_y column and rename datetime_x to datetime
complete_data = complete_data.drop(columns=['datetime_y'])
complete_data = complete_data.rename(columns={'datetime_x': 'datetime'})

In [20]:
# print columns
print(complete_data.dtypes)

datetime               datetime64[ns]
violation_year                  int32
violation_month                 int32
violation_day_month             int32
violation_day_week              int32
violation_hour                  int32
violation_count                 int64
dtype: object


## Augment Data

In [21]:
# Define the file path pattern to read all CSV files (adjust the path as needed)
file_pattern = '../task2-aug/nyc_w_*.csv'

# Specify the data types for each column
dtypes = {
    'datetime': 'str',  # 'datetime' will be parsed separately
    'temp': 'float64',
    'feelslike': 'float64',
    'dew': 'float64',
    'humidity': 'float64',
    'precip': 'float64',
    'precipprob': 'float64',
    'preciptype': 'object',  # String data type
    'snow': 'float64',
    'snowdepth': 'float64',
    'windgust': 'float64',
    'windspeed': 'float64',
    'winddir': 'float64',
    'sealevelpressure': 'float64',
    'cloudcover': 'float64',
    'visibility': 'float64',
    'solarradiation': 'float64',
    'solarenergy': 'float64',
    'uvindex': 'float64',
    'severerisk': 'float64',
    'conditions': 'object',  # String data type
    'icon': 'object',  # String data type
    'stations': 'object'  # String data type
}

# Read all CSV files into a single Dask DataFrame with specified dtypes
df = dd.read_csv(file_pattern, dtype=dtypes)

# Convert the 'datetime' column to datetime type
df['datetime'] = dd.to_datetime(df['datetime'], format='%Y-%m-%dT%H:%M:%S', errors='coerce')

# Drop all string columns except 'datetime'
columns_to_keep = ['datetime'] + [col for col in df.columns if df[col].dtype in ['float64', 'int64']]
df = df[columns_to_keep]





In [22]:
# print types of the columns
print(df.dtypes)

datetime            datetime64[ns]
temp                       float64
feelslike                  float64
dew                        float64
humidity                   float64
precip                     float64
precipprob                 float64
snow                       float64
snowdepth                  float64
windgust                   float64
windspeed                  float64
winddir                    float64
sealevelpressure           float64
cloudcover                 float64
visibility                 float64
solarradiation             float64
solarenergy                float64
uvindex                    float64
severerisk                 float64
dtype: object


## Join Data

In [23]:
# Merge the daily data with the weather data
augmented_data = data.merge(df, left_on='datetime', right_on='datetime', how='left')

# print(augmented_data.head())

In [ ]:
# count the number of missing values in each column
print(augmented_data.isnull().sum().compute())

In [25]:
# drop sewerisk and windgust column
augmented_data = augmented_data.drop(columns=['severerisk', 'windgust'])

# drop all rows with missing values
augmented_data = augmented_data.dropna()


In [ ]:
augmented_data.to_parquet('augmented_data', write_index=False)